# Initialization and Tests

In [1]:
1

1

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append("/home/icb/kemal.inecik/work/codes/idtrack")

import os
import time
import pickle
import idtrack
import copy

In [4]:
local_dir = "/lustre/groups/ml01/workspace/kemal.inecik/idtrack_temp/"
idt = idtrack.API(local_repository=local_dir)
idt.configure_logger()
idt.get_ensembl_organism("homo sapiens")

2025-05-30 12:23:51 INFO:verify_organism: Ensembl Rest API query to get the organism names and associated releases.


('homo_sapiens', 114)

In [5]:
!ls -l /lustre/groups/ml01/workspace/kemal.inecik/idtrack_temp/

total 42110132
-rw-r--r--. 1 kemal.inecik OG-ICB-User   852126351 May 27 20:30 graph_homo_sapiens_min79_max114_narrow.pickle
-rw-r--r--. 1 kemal.inecik OG-ICB-User 21376563160 May 27 20:28 homo_sapiens_assembly-37.h5
-rw-r--r--. 1 kemal.inecik OG-ICB-User 20891831455 May 27 20:28 homo_sapiens_assembly-38.h5
-rw-r--r--. 1 kemal.inecik OG-ICB-User       65514 May 27 17:59 homo_sapiens_externals_modified.yml
-rw-r--r--. 1 kemal.inecik OG-ICB-User       55077 May 27 17:26 homo_sapiens_externals_template.yml


In [6]:
idt.initialize_graph(organism_name='homo_sapiens', last_ensembl_release=114, return_test=True)

2025-05-30 12:23:53 INFO:graph_maker: The graph is being read: /lustre/groups/ml01/workspace/kemal.inecik/idtrack_temp/graph_homo_sapiens_min79_max114_narrow.pickle


In [7]:
idt.calculate_graph_caches()

2025-05-30 12:24:45 INFO:the_graph: Cached properties being calculated: combined_edges
2025-05-30 12:25:29 INFO:the_graph: Cached properties being calculated: combined_edges_assembly_specific_genes
2025-05-30 12:25:56 INFO:the_graph: Cached properties being calculated: combined_edges_genes
2025-05-30 12:26:12 INFO:the_graph: Cached properties being calculated: lower_chars_graph
2025-05-30 12:26:12 INFO:the_graph: Cached properties being calculated: get_active_ranges_of_id
2025-05-30 12:26:35 INFO:the_graph: Cached properties being calculated: available_external_databases
2025-05-30 12:26:36 INFO:the_graph: Cached properties being calculated: external_database_connection_form
2025-05-30 12:27:22 INFO:the_graph: Cached properties being calculated: available_genome_assemblies
2025-05-30 12:27:23 INFO:the_graph: Cached properties being calculated: available_external_databases_assembly
2025-05-30 12:27:24 INFO:the_graph: Cached properties being calculated: node_trios


#### Structural Tests

In [7]:
idt.track.is_id_functions_consistent_ensembl()

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [01:17<00:00,  2.14s/it, Item:114]


True

In [8]:
idt.track.is_id_functions_consistent_ensembl_2()

100%|█████████████████████████████████████████████████████████████████████████████| 165093/165093 [00:00<00:00, 176122.22it/s, Item:LRG_99.1]


True

In [9]:
idt.track.is_range_functions_robust()

100%|██████████████████████████████████████████████████████████████████████| 100842/100842 [00:00<00:00, 239189.69it/s, Item:ENSG00000169933]


True

In [10]:
idt.track.is_base_is_range_correct()

100%|███████████████████████████████████████████████████████████████████████| 100842/100842 [00:03<00:00, 29035.43it/s, Item:ENSG00000169933]


True

In [11]:
idt.track.is_combined_edges_dicts_overlapping_and_complete()

True

In [12]:
idt.track.is_edge_with_same_nts_only_at_backbone_nodes()

True

In [14]:
idt.track.is_node_consistency_robust()

100%|██████████████████████████████████████████████████████████████████████████████████████████| 2507048/2507048 [00:08<00:00, 309828.55it/s]


True

In [20]:
idt.track.is_id_functions_consistent_external()

Assembly 38: 100%|█████████████████████████████████████████████████████████████████████████████████| 36/36 [34:12<00:00, 57.00s/it, Item:114]


True

#### Conversion Tests

In [98]:
for i in range(1):
    r = idt.track.history_travel_testing_random(from_fraction=0.05, include_external_destination=False, return_result=True)

2025-05-30 17:23:50 WARNING:track_tests: Recalculating parameters.
100%|███████████████████████████████████████████████████████████████| 3401/3401 [04:46<00:00, 11.89it/s, Item:LRG_887]
2025-05-30 17:28:57 INFO:track_tests: 
╔═ History-Travel-Testing Report ═╗
Source  : base_ensembl_gene (Assembly 38, Release 100)
Target  : base_ensembl_gene (Release 85)
External: True   1→1-pref.: True
Sample  : 0.05 of source IDs

Failure / Anomaly Counts:
  - Voyage failed (graceful) : 0
  - Voyage failed (unknown)  : 0
  - Query not in graph       : 0
  - Lost item                : 112
  - Found IDs not accurate   : 0

Mapping Statistics:
  - One→one IDs     : 3,276
  - One→many IDs    : 13
  - Clash one→one   : 31
  - Clash many→many : 4
  - Clash mixed     : 1

Total runtime: 286.18 s


In [100]:
for i in range(5):
    r = idt.track.history_travel_testing_random(from_fraction=0.05, return_result=True)

100%|████████████████████████████████████████████████████████████████| 1290/1290 [00:19<00:00, 66.75it/s, Item:stxbp1]
2025-05-30 17:32:04 INFO:track_tests: 
╔═ History-Travel-Testing Report ═╗
Source  : NCBI gene (formerly Entrezgene) (Assembly 37, Release 112)
Target  : Uniprot/SWISSPROT (Release 92)
External: True   1→1-pref.: True
Sample  : 0.05 of source IDs

Failure / Anomaly Counts:
  - Voyage failed (graceful) : 331
  - Voyage failed (unknown)  : 0
  - Query not in graph       : 0
  - Lost item                : 3
  - Found IDs not accurate   : 0

Mapping Statistics:
  - One→one IDs     : 949
  - One→many IDs    : 7
  - Clash one→one   : 5
  - Clash many→many : 0
  - Clash mixed     : 0

Total runtime: 19.33 s
100%|██████████████████████████████████████████████████████████████████| 955/955 [00:14<00:00, 64.90it/s, Item:Q9Y6U7]
2025-05-30 17:32:39 INFO:track_tests: 
╔═ History-Travel-Testing Report ═╗
Source  : Uniprot/SWISSPROT (Assembly 37, Release 87)
Target  : Clone_based_veg

KeyboardInterrupt: 

In [101]:
to_assembly

NameError: name 'to_assembly' is not defined

In [47]:
r = idt.track.history_travel_testing(
    from_release = 87,
    from_assembly = 37,
    from_database = "Uniprot/SWISSPROT",
    to_release = 83,
    to_database = "Clone_based_vega_gene",
    go_external = True,
    prioritize_to_one_filter = True,
    convert_using_release = False,
    from_fraction = 0.01,
    verbose = True,
    verbose_detailed = False,
    return_ensembl_alternative = False,
)

100%|████████████████████████████████████████████████████████████| 567/567 [00:13<00:00, 42.58it/s, Item:XM_047449851]


In [53]:
print(r["history_voyage_failed_gracefully"][0][1])

Traceback (most recent call last):
  File "/home/icb/kemal.inecik/work/codes/idtrack/idtrack/_track_tests.py", line 591, in history_travel_testing
    converted_item = self.convert(
                     ^^^^^^^^^^^^^
  File "/home/icb/kemal.inecik/work/codes/idtrack/idtrack/_track.py", line 1816, in convert
  File "/home/icb/kemal.inecik/work/codes/idtrack/idtrack/_track.py", line 2139, in _final_conversion
    min_priority_list=sl2,
idtrack._db.EmptyConversionMetricsError: The `conversion_metrics` is emtpy. It happens when `return_ensembl_alternative` is `false` and no corresponding final conversion possible: ENSG00000163395.17
{'ENSG00000163395.17': {'from_id': 'XM_005245580', 'assembly_jump': 0, 'external_jump': 1, 'external_step': 2, 'initial_conversion_conf': 1, 'edge_scores_reduced': nan, 'ensembl_step': 0, 'final_assembly_priority': ([1], 1)}}.



In [54]:
r = idt.track.history_travel_testing(
    from_release = 86,
    from_assembly = 37,
    from_database = "Clone_based_vega_gene",
    to_release = 111,
    to_database = "assembly_38_ensembl_translation",
    go_external = True,
    prioritize_to_one_filter = True,
    convert_using_release = False,
    from_fraction = 0.01,
    verbose = True,
    verbose_detailed = False,
    return_ensembl_alternative = False,
)

100%|███████████████████████████████████████████████████████| 198/198 [00:03<00:00, 65.39it/s, Item:XXbac-BPG181B23.6]


In [56]:
print(idt.track.format_history_travel_testing_report(r))


╔═ History-Travel-Testing Report ═╗
Source  : Clone_based_vega_gene (Assembly 37, Release 86)
Target  : assembly_38_ensembl_translation (Release 111)
External: True   1→1-pref.: True
Sample  : 0.01 of source IDs

Failure / Anomaly Counts:
  - Voyage failed (graceful) : 0
  - Voyage failed (unknown)  : 191
  - Query not in graph       : 0
  - Lost item                : 7
  - Found IDs not accurate   : 0

Mapping Statistics:
  - One→one IDs     : 0
  - One→many IDs    : 0
  - Clash one→one   : 0
  - Clash many→many : 0
  - Clash mixed     : 0

Total runtime: 3.14 s


In [59]:
print(r["history_voyage_failed_unknown"][0][0])

AC000077.2


In [103]:
idt.track.graph.available_external_databases_assembly[37]

{'CCDS',
 'Clone_based_ensembl_gene',
 'Clone_based_vega_gene',
 'EntrezGene',
 'HGNC Symbol',
 'Havana gene',
 'Havana transcript',
 'NCBI gene (formerly Entrezgene)',
 'RFAM',
 'RefSeq_mRNA',
 'RefSeq_mRNA_predicted',
 'RefSeq_ncRNA',
 'RefSeq_ncRNA_predicted',
 'RefSeq_peptide',
 'RefSeq_peptide_predicted',
 'UniProtKB Gene Name',
 'Uniprot/SPTREMBL',
 'Uniprot/SWISSPROT',
 'synonym_id::EntrezGene',
 'synonym_id::HGNC Symbol',
 'synonym_id::NCBI gene (formerly Entrezgene)',
 'synonym_id::UniProtKB Gene Name'}

In [104]:
idt.track.graph.available_external_databases_assembly[38]

{'CCDS',
 'Clone_based_ensembl_gene',
 'Clone_based_vega_gene',
 'EntrezGene',
 'HGNC Symbol',
 'Havana gene',
 'Havana transcript',
 'Havana translation',
 'NCBI gene',
 'NCBI gene (formerly Entrezgene)',
 'RFAM',
 'RefSeq_mRNA',
 'RefSeq_mRNA_predicted',
 'RefSeq_ncRNA',
 'RefSeq_ncRNA_predicted',
 'RefSeq_peptide',
 'RefSeq_peptide_predicted',
 'UniProtKB Gene Name',
 'Uniprot/SPTREMBL',
 'Uniprot/SWISSPROT',
 'Vega gene',
 'Vega_gene',
 'synonym_id::EntrezGene',
 'synonym_id::HGNC Symbol',
 'synonym_id::NCBI gene',
 'synonym_id::NCBI gene (formerly Entrezgene)',
 'synonym_id::UniProtKB Gene Name'}

In [106]:
idt.track.graph.available_releases_given_database_assembly("Clone_based_vega_gene", 38)

{79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89}

In [72]:
idt.track.convert("AC000077.2", to_release=111, from_release=None, final_database="assembly_38_ensembl_translation", return_path=True)

assembly_38_ensembl_translation
{'Havana transcript', 'synonym_id::HGNC Symbol', 'RefSeq_mRNA_predicted', 'Havana gene', 'synonym_id::NCBI gene', 'synonym_id::EntrezGene', 'RefSeq_mRNA', 'Uniprot/SPTREMBL', 'synonym_id::NCBI gene (formerly Entrezgene)', 'RefSeq_peptide_predicted', 'Vega_gene', 'CCDS', 'Havana translation', 'RefSeq_peptide', 'NCBI gene', 'synonym_id::UniProtKB Gene Name', 'Clone_based_vega_gene', 'UniProtKB Gene Name', 'Clone_based_ensembl_gene', 'RefSeq_ncRNA_predicted', 'RefSeq_ncRNA', 'NCBI gene (formerly Entrezgene)', 'Vega gene', 'HGNC Symbol', 'Uniprot/SWISSPROT', 'EntrezGene', 'RFAM'}
base_ensembl_gene


ValueError: for cnvt in converted:

In [66]:
from idtrack import DB
DB.nts_ensembl

{'gene': 'ensembl_gene',
 'transcript': 'ensembl_transcript',
 'translation': 'ensembl_translation'}

In [83]:
idt.track.graph.external_database_connection_form

{'Havana transcript': 'transcript',
 'synonym_id::HGNC Symbol': 'gene',
 'RefSeq_mRNA_predicted': 'transcript',
 'Havana gene': 'gene',
 'synonym_id::NCBI gene': 'gene',
 'synonym_id::EntrezGene': 'gene',
 'RefSeq_mRNA': 'transcript',
 'Uniprot/SPTREMBL': 'translation',
 'synonym_id::NCBI gene (formerly Entrezgene)': 'gene',
 'RefSeq_peptide_predicted': 'translation',
 'Vega_gene': 'gene',
 'CCDS': 'transcript',
 'Havana translation': 'translation',
 'RefSeq_peptide': 'translation',
 'NCBI gene': 'gene',
 'synonym_id::UniProtKB Gene Name': 'gene',
 'Clone_based_vega_gene': 'gene',
 'UniProtKB Gene Name': 'gene',
 'Clone_based_ensembl_gene': 'gene',
 'RefSeq_ncRNA_predicted': 'transcript',
 'RefSeq_ncRNA': 'transcript',
 'NCBI gene (formerly Entrezgene)': 'gene',
 'Vega gene': 'gene',
 'HGNC Symbol': 'gene',
 'Uniprot/SWISSPROT': 'translation',
 'EntrezGene': 'gene',
 'RFAM': 'gene'}

In [71]:
DB.nts_base_ensembl

{'gene': 'base_ensembl_gene',
 'transcript': 'base_ensembl_transcript',
 'translation': 'base_ensembl_translation'}

In [74]:
DB.nts_ensembl["gene"]

'ensembl_gene'

In [75]:
DB.nts_base_ensembl["gene"]

'base_ensembl_gene'

In [85]:
list(DB.nts_assembly[38].values())

['assembly_38_ensembl_gene',
 'assembly_38_ensembl_transcript',
 'assembly_38_ensembl_translation']

In [87]:
DB.nts_assembly[38]["gene"]

'assembly_38_ensembl_gene'

In [90]:
DB.nts_ensembl

{'gene': 'ensembl_gene',
 'transcript': 'ensembl_transcript',
 'translation': 'ensembl_translation'}

In [ ]:
for _ in range(10):
    idt.track.history_travel_testing_random(from_fraction=0.1)

In [ ]:
for _ in range(10):
    idt.track.history_travel_testing_random(from_fraction=0.1)
    response, _ = idt.track.is_external_conversion_robust(database="RFAM", ens_rel=94)
    print(response)

In [17]:
for _ in range(10)
r, _ = idt.track.is_external_conversion_robust()
r

Random database and Ensembl release.
Assembly: 38, Database: Vega_gene, Release: 83


100%|███████████████████████████████████████████████████████████| 66858/66858 [05:13<00:00, 213.49it/s, Item:LRG_99.1]


True

#### Conversion Tests in Depth

In [9]:
response, (issues_t1, issues_t2, issues_t3, res) = idt.track.is_external_conversion_robust(
    database="RFAM",
    ens_rel=94
)

Assembly: 38, Database: RFAM, Release: 94


100%|██████████████████████████████████████████████████████████| 65687/65687 [06:08<00:00, 178.33it/s, Item:LRG_999.1]


In [10]:
dm_original = idt.track.db_manager
dm = dm_original.change_release(94).change_assembly(38)
df_a = dm.get_db("external_relevant")
df = df_a[df_a["name_db"] == "RFAM"].copy()

base_dict: dict[str, set] = dict()
for _, item in df.iterrows():
    if item["graph_id"] not in base_dict:
        base_dict[item["graph_id"]] = set()
    base_dict[item["graph_id"]].add(item["id_db"])

In [19]:
res=r

In [11]:
response, len(res["conversion"]), len(issues_t1), len(issues_t2), len(issues_t3)

(True, 5612, 0, 0, 60075)

In [12]:
len(df[df["graph_id"].isin(issues_t3)]) == 0

True

In [20]:
(
    len(res["history_voyage_failed_gracefully"]),
    len(res["history_voyage_failed_unknown"]),
    len(res["query_not_in_the_graph"]),
    len(res['lost_item']),
    len(res['found_ids_not_accurate'])
)

(
    len(res['one_to_one_ids']),
    len(res['one_to_multiple_ids']),
    res["clashing_id_type"]
)

(1961, 0, 0, 2, 0)

In [15]:
idt.convert_identifier("ENSG00000199293.1", to_release=94, from_release=94, final_database="RFAM", return_path=True)

{'target_id': ['RF00412'],
 'last_node': [('ENSG00000199293.1', 'RF00412')],
 'final_database': 'RFAM',
 'graph_id': 'ENSG00000199293.1',
 'query_id': 'ENSG00000199293.1',
 'no_corresponding': False,
 'no_conversion': False,
 'no_target': False,
 'the_path': {('RF00412',
   'ENSG00000199293.1'): ((None,
    'ENSG00000199293.1',
    None), ('ENSG00000199293.1', 'RF00412', 0, 94))}}

In [16]:
idt.track.convert("ENSG00000199293.1", to_release=94, from_release=94, final_database="RFAM", return_path=True)

{'ENSG00000199293.1': {'from_id': 'ENSG00000199293.1',
  'assembly_jump': 0,
  'external_jump': 0,
  'external_step': 0,
  'initial_conversion_conf': 1,
  'edge_scores_reduced': nan,
  'ensembl_step': 1,
  'final_assembly_priority': ([1, 2], 2),
  'the_path': ((None, 'ENSG00000199293.1', None),),
  'final_conversion': {'final_conversion_confidence': 0,
   'final_database': 'RFAM',
   'final_elements': {'RF00412': {'final_assembly_priority_count': 2,
     'final_assembly_min_priority': 1,
     'additional_assembly_jump': 0,
     'the_path': (('ENSG00000199293.1', 'RF00412', 0, 94),)}}}}}